In [ ]:
import pandas as pd
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
import os
import cv2
import sys
import math
from pathlib import Path
from compress_pickle import dump, load

PIPELINE_ROOT = Path('./').resolve().parent.parent.parent
sys.path.append(PIPELINE_ROOT.as_posix())
print(PIPELINE_ROOT)

from src.library.utilities.utilities_process import read_image, write_image
from src.library.cell_labeling.cell_utilities import calculate_correlation_and_energy, find_connected_segments, load_image, subtract_blurred_image


In [ ]:
def show_image(img, increaser=1, _title="", fsize=[15,15]):
    plt.figure(figsize=[15,15])
    plt.imshow(img * increaser, cmap="gray")
    plt.title(_title,fontsize=20)
    plt.tick_params(axis='x', labelsize=20)
    plt.tick_params(axis='y', labelsize=20)
    plt.show()

def collect_positives(uimg, startr, startc, tile_width, tile_height):
    img=np.copy(uimg)
    positives=[]
    radius = 80
    a=int(radius/2)
    color = (65000)
    min_row,min_col = startr, startc
    for index,r in df.iterrows():
        colr = r['x']
        rowr = r['y']
        col = int(round(r['x'])-min_col)
        row = int(round(r['y'])-min_row)
        print(col, row)
        if (col<0) or (col>tile_width) or (row<0) or (row>tile_height):
            continue
        img=cv2.circle(img, (int(col), int(row)), radius, color, 20)
        I=uimg[row-a:row+a,col-a:col+a]
        if I.shape==(radius,radius):
            positives.append((col,row,I, colr, rowr))

    return img,positives

In [ ]:
animal = 'DK184'
section = 0
channel = 3
prep_path = f'/net/birdstore/Active_Atlas_Data/data_root/pipeline_data/{animal}/preps'

In [ ]:
tif = str(section).zfill(3) + ".tif"
filepath = os.path.join(prep_path, f'C{channel}/full_aligned', tif)
full_img = read_image(filepath)
full_img.shape

In [ ]:
resize = True
if resize:
    filepath = os.path.join(prep_path, f'C{channel}/full_aligned', tif)
    startr = 0
    endr = 25000
    startc = 35000
    endc = 65000
    img = full_img[startr:endr, startc:endc]
    print(img.shape)
    write_image(filepath, img)

In [ ]:
dfpath = os.path.join(prep_path, 'cell_labels', 'all_predictions.csv')
df = pd.read_csv(dfpath)
df = df.loc[df['section'] == section]
df = df.sort_values(['section', 'y', 'x'])
df.head()

In [ ]:
dfp.loc[dfp['x'] == col]

In [ ]:
dfpath = os.path.join(prep_path, 'cell_labels', 'detections_000.csv')
df = pd.read_csv(dfpath)
#bads = df[( ( (df['row'] < 8000) | (df['row'] > 22752 ))  & (df['predictions']) == 0 )]
#len(bads.index)

In [ ]:
#bads = df[(df['row'] > 8000) & (df['col'] < 27000 )  & (df['area'] > 100) & (df['col'] > 18000) ]

df.head()
#df.loc[df['col'] == col]

In [ ]:
#nos = df[(df['predictions'] == -2 )]
#unsures = df[(df['predictions'] == 0 )]
#sures = df[(df['predictions'] == 2 )]
#print(f'nos={len(nos)} unsures={len(unsures)} sures={len(sures)}')

In [ ]:
# set bad Y prediction to bad
for index, df_row in df.iterrows():
    row = int(df_row['row'])
    col = int(df_row['col'])
    prediction = df_row['predictions']
    found = row < 8000 or row > 22752
    if found:
        #print(f'ERROR: Predicted cell {index=} found at {row=}, {col=} {prediction=}')
        df.loc[index, 'predictions'] = -2




nos = df[(df['predictions'] == -2 )]
unsures = df[(df['predictions'] == 0 )]
sures = df[(df['predictions'] == 2 )]
print(f'nos={len(nos)} unsures={len(unsures)} sures={len(sures)}')

In [ ]:
dfpath = os.path.join(prep_path, 'cell_labels', 'better_detections_000.csv')
df.to_csv(dfpath, index=False)

In [ ]:
print(df.head(3))
print(df.tail(3))

In [ ]:
count = len(df.index)
print(f'{count=}')
if count > 100:
    lower = count // 4
    upper = lower + 100
    print(lower, upper)
    df = df[lower:upper]
print(len(df.index))

In [ ]:
#startc = int(round(min(df['x']), -3)) - 1000
#startr = int(round(min(df['y']), -3)) - 1000
#endc = math.ceil(max(df['x'])) + 100
#endr = math.ceil(max(df['y'])) + 100
# manually set to ROI
startr = 0
endr = startr + full_img.shape[0]
startc = 0
endc = startc + full_img.shape[1]
print(startr, endr, startc, endc)

In [ ]:
tile_width = endc - startc
tile_height = endr - startr
img = full_img[startr:endr, startc:endc]
print(f'{startr}:{endr}, {startc}:{endc}')
print(img.shape)
print(f'{tile_width=} {tile_height=}')
#img.shape # (6800, 30000)

In [ ]:
annotated_img, positives = collect_positives(full_img, startr, startc, tile_width, tile_height)
print(len(positives))

In [ ]:
#vimg=annotated_img[in_row_min:in_row_max, in_col_min: in_col_max]
increaser = 1
show_image(annotated_img, increaser=increaser,_title=f'{animal} C{channel}, section: {section} predictions: {len(positives)}')

In [ ]:
plt.figure(figsize=[15,15])
end = min(157, len(positives))
for i in range(len(positives[0:end])):
    plt.subplot(16,10,i+1)
    plt.title(i)
    plt.imshow(positives[i][2], cmap="gray")
plt.show()

In [ ]:
S=np.stack([I for (x,y,I)in positives])
S.shape

Mean=np.mean(S,axis=0)
Mean.shape

plt.imshow(Mean, cmap="gray")
plt.show()

_mean=np.mean(Mean.flatten())
_std=np.std(Mean.flatten())
print('before: mean=%5.2f std=%5.2f'%(_mean,_std))

Mean = (Mean-_mean)/_std

_mean=np.mean(Mean.flatten())
_std=np.std(Mean.flatten())
print('after: mean=%5.2f std=%5.2f'%(_mean,_std))

import pickle as pk
with open('mean.pkl','wb') as pkl_file:
    pk.dump({'Mean':Mean},pkl_file)

In [ ]:
new_mean = cv2.normalize(Mean, None, 255, 0, cv2.NORM_MINMAX, cv2.CV_8U)
plt.imshow(new_mean, cmap="seismic")
plt.show()

In [ ]:
tif = str(section).zfill(3) + ".tif"
filepath = os.path.join(prep_path, f'C{channel}/full_aligned', tif)
input_file_virus_path = os.path.join(prep_path, 'C3/full_aligned', tif)  
input_file_dye_path = os.path.join(prep_path, 'C1/full_aligned', tif)  
data_virus = read_image(input_file_virus_path)
data_dye = read_image(input_file_dye_path)
print(f'{data_virus.shape=} {data_dye.shape=}')

In [ ]:
avg_cell_img_file_path = '/net/birdstore/Active_Atlas_Data/cell_segmentation/models/average_cell_image.pkl'
avg_cell_img = load(avg_cell_img_file_path) #Load average cell image once
avg_cell_CH1 = avg_cell_img['CH1']
avg_cell_CH3 = avg_cell_img['CH3']

print(avg_cell_CH1.shape)

In [ ]:
cell = 0
col = positives[cell][1]
row = positives[cell][0]
colr = positives[cell][4]
rowr = positives[cell][3]
control_img = positives[cell][2]
section = 0
print(f'processing coordinates {col=}, {row=}, {section=}')
print(f'processing coordinates {colr=}, {rowr=}, {section=}')
idx = 0

startc = col - 40
endc = startc + 80
startr = row - 40
endr = startr + 80

print(f'{startr=} {endr=} {startc=} {endc=}')

In [ ]:
image_roi_virus = data_virus[startc:endc, startr:endr] #image_roi IS numpy array
image_roi_dye = data_dye[startc:endc, startr:endr] #image_roi IS numpy array
print(f'shape of image_roi_virus {image_roi_virus.shape} and shape of data_virus {image_roi_dye.shape}')
absolute_coordinates = (startc, endc, startr, endr)
difference_ch1 = subtract_blurred_image(image_roi_dye, make_smaller=False) #calculate img difference for virus channel (e.g. fluorescence)
difference_ch3 = subtract_blurred_image(image_roi_virus, make_smaller=False) #calculate img difference for virus channel (e.g. fluorescence)


In [ ]:
print(f'shape of data_dye {data_dye.shape}')
print(f'{row=} {col=} {startr=} {endr=} {startc=} {endc=}')
f, axarr = plt.subplots(2,2)
axarr[0,0].imshow(image_roi_dye, cmap="gray")
axarr[0,1].imshow(control_img, cmap="gray")
axarr[1,0].imshow(avg_cell_CH1, cmap="gray")
axarr[1,1].imshow(difference_ch1, cmap="gray")


In [ ]:
connected_segments = find_connected_segments(difference_ch3, 2000)
n_segments, segment_masks, segment_stats, segment_location = (connected_segments)
print(f'Found {n_segments} segments')
print(f'{segment_stats=}')
print(f'{segment_location=}')
print(f'difference_ch3 shape {difference_ch3.shape}')
print(f'shape of segmentmasks {segment_masks.shape}')
ids, counts = np.unique(segment_masks, return_counts=True)
print(len(ids))
print(len(counts))


In [ ]:
plt.imshow(segment_masks, cmap="gray")
plt.show()

In [ ]:
segmenti = 0
_, _, width, height, object_area = segment_stats[segmenti, :]
print(f'{width=} {height=} {object_area=}')


In [ ]:
segment_row, segment_col = segment_location[segmenti, :]
print(f'{segment_row=}, {segment_col=}, {width=}, {height=}, {object_area=}')

In [ ]:
cell_radius = 40
row_start = int(segment_row - cell_radius)
col_start = int(segment_col - cell_radius)
row_end = int(segment_row + cell_radius)
col_end = int(segment_col + cell_radius)
segment_mask = (segment_masks[row_start:row_end, col_start:col_end] == segmenti)
print(row_start, row_end, col_start, col_end)
print(f'segment mask {segment_mask.shape}')

In [ ]:
ch1_corr, ch1_energy = calculate_correlation_and_energy(avg_cell_img["CH1"], difference_ch1.T)
print(ch1_corr, ch1_energy)
#plt.imshow(segment_masks, cmap="gray")
#plt.show()

In [ ]:
animal: DK184
section: 0
index: 45202
row: 5718
col: 26346
area: 3575
height: 68
width: 71
corr_CH1: -0.062213653117781466
energy_CH1: 0.03439795782711712
corr_CH3: 1.5275977775282754
energy_CH3: 0.21030696615058242
m00_mask: 3575.0
m10_mask: 144022.0
m01_mask: 143666.0
m20_mask: 6906752.0
m11_mask: 6009085.0
m02_mask: 6756562.0
m30_mask: 367217452.0
m21_mask: 295736537.0
m12_mask: 290128787.0
m03_mask: 350298704.0
mu20_mask: 1104699.836643357
mu11_mask: 221374.6078321682
mu02_mask: 983157.9283916088
mu30_mask: -34686.82240531219
mu21_mask: 343233.39988559886
mu12_mask: 142330.72541748337
mu03_mask: -241427.5168160958
nu20_mask: 0.08643550973785374
nu11_mask: 0.017321109713505263
nu02_mask: 0.07692565335354169
nu30_mask: -4.5391486079870513e-05
nu21_mask: 0.0004491582973788852
nu12_mask: 0.00018625526045695513
nu03_mask: -0.0003159342081209841
h0_mask: 0.16336116309139542
h1_mask: 0.001290520735279386
h2_mask: 3.1319358386334703e-06
h3_mask: 3.759126089035539e-08
h4_mask: 1.2101273526074454e-14
h5_mask: 1.320135938455445e-09
h6_mask: -4.464147373450296e-15
contrast1: -0.033574577420949936
contrast3: 0.2903658449649811